In this practice code, we are going to use Ozone dataset. Dictionary of this dataset can be found in https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/airquality.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
url = "/Users/jcano/src/repos/SF-DAT-20/Data/ozone.csv"
OzoneData = pd.read_csv(url)

#### Explore the dataset and decide which variables suffer from missing data

In [3]:
OzoneData.describe()

,Ozone,Solar.R,Wind,Temp,Month,Day
count,116.000000,146.000000,153.000000,153.000000,153.000000,153.000000
mean,42.129310,185.931507,9.957516,77.882353,6.993464,15.803922
std,32.987885,90.058422,3.523001,9.465270,1.416522,8.864520
min,1.000000,7.000000,1.700000,56.000000,5.000000,1.000000
25%,18.000000,115.750000,7.400000,72.000000,6.000000,8.000000
50%,31.500000,205.000000,9.700000,79.000000,7.000000,16.000000
75%,63.250000,258.750000,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


Answer: Ozone and Solar.R are missing some data.

#### Let's drop rows that have missing values in all the columns you indentified above

Hint: in dropna() if you set how = 'all', it will only drop columns that are suffering from missing values at all varaibles you introduce in subset. If you want to get rid of the row that contains missing values in any of the variables you specify, then you shall set how = 'any'

df.dropna(how = 'all',subset = ['Var1','Var2','Var3'],inplace = True)

The above code will check if all 3 variables specified in df have missing values, if they all have missing values it will drop that row.

In [4]:
OzoneData.dropna(how="all", subset=["Ozone", "Solar.R"], inplace=True)

In [5]:
OzoneData.describe()

,Ozone,Solar.R,Wind,Temp,Month,Day
count,116.000000,146.000000,151.000000,151.000000,151.000000,151.000000
mean,42.129310,185.931507,9.941722,78.165563,7.019868,15.801325
std,32.987885,90.058422,3.524984,9.198138,1.406984,8.832531
min,1.000000,7.000000,1.700000,57.000000,5.000000,1.000000
25%,18.000000,115.750000,7.400000,73.000000,6.000000,8.000000
50%,31.500000,205.000000,9.700000,79.000000,7.000000,16.000000
75%,63.250000,258.750000,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


#### Spoiler! If everything is going according to plan you shall be left by 151 observations. Also, it seemed like the first two variables had missing values. Now plese make a copy of your dataframe into a dataframe named OzoneImputeMean. Also, please use mean of the variables to fill in missing values in OzoneImputeMean

In [6]:

OzoneImputeMean = OzoneData.copy()
OzoneMissing = OzoneData.dropna(how="any", subset=["Ozone"], inplace=False)
OzoneImputeMean["Ozone"].fillna(value=OzoneMissing["Ozone"].mean(), inplace=True)
OzoneMissing = OzoneData.dropna(how="any", subset=["Solar.R"], inplace=False)
OzoneImputeMean["Solar.R"].fillna(value=OzoneMissing["Solar.R"].mean(), inplace=True)

In [7]:
#Check if missing values are filled
OzoneImputeMean.describe()


,Ozone,Solar.R,Wind,Temp,Month,Day
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,42.129310,185.931507,9.941722,78.165563,7.019868,15.801325
std,28.884028,88.544727,3.524984,9.198138,1.406984,8.832531
min,1.000000,7.000000,1.700000,57.000000,5.000000,1.000000
25%,21.000000,119.000000,7.400000,73.000000,6.000000,8.000000
50%,42.129310,197.000000,9.700000,79.000000,7.000000,16.000000
75%,46.500000,257.000000,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


#### Now it's time for imputing using linear regression lines

In [8]:
#### Before we start let's define dummy variables for variable Month - don't worry about day!
MonthDummy = pd.get_dummies(OzoneData.Month, prefix = 'Month')
del MonthDummy['Month_9']
OzoneData = pd.concat([OzoneData, MonthDummy], axis=1)
OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,41,190,7.4,67,5,1,1,0,0,0
1,36,118,8.0,72,5,2,1,0,0,0
2,12,149,12.6,74,5,3,1,0,0,0
3,18,313,11.5,62,5,4,1,0,0,0
5,28,NaN,14.9,66,5,6,1,0,0,0


In [9]:
#now let's explore correlation Matrix
OzoneData.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.348342,-0.601547,0.698360,0.164519,-0.013226,-0.302963,-0.112006,0.277961,0.291808
Solar.R,0.348342,1.000000,-0.056792,0.275840,-0.075301,-0.150275,-0.024601,0.023998,0.176744,-0.076390
Wind,-0.601547,-0.056792,1.000000,-0.466032,-0.175317,0.042365,0.237781,0.046054,-0.144638,-0.166105
Temp,0.698360,0.275840,-0.466032,1.000000,0.397427,-0.136876,-0.637816,0.050753,0.318103,0.321680
Month,0.164519,-0.075301,-0.175317,0.397427,1.000000,-0.007727,-0.702257,-0.362131,-0.007201,0.355246
Day,-0.013226,-0.150275,0.042365,-0.136876,-0.007727,1.000000,0.011003,-0.017044,0.011471,0.011471
Month_5,-0.302963,-0.024601,0.237781,-0.637816,-0.702257,0.011003,1.000000,-0.242766,-0.247805,-0.247805
Month_6,-0.112006,0.023998,0.046054,0.050753,-0.362131,-0.017044,-0.242766,1.000000,-0.253080,-0.253080
Month_7,0.277961,0.176744,-0.144638,0.318103,-0.007201,0.011471,-0.247805,-0.253080,1.000000,-0.258333
Month_8,0.291808,-0.076390,-0.166105,0.321680,0.355246,0.011471,-0.247805,-0.253080,-0.258333,1.000000


#### What seems to be the list of best variables can define Ozone? how about Solar.R?

Answer: IT seems that Temp, Wind and Solar.R are the best variables to use.

In [10]:
# now let's use a regression model to predict Ozone. First drop NaN values in Ozone and save it in OzoneDroppedValues_Ozone
# the run a regression line on variables of interest and check significancy of your model
# if in a multi class dummy variable case, you see a few of the dummy variables are not significant but the rest are, 
#you shall either drop all or keep all. Otherwise, selecting the base dummy will become important
# use these variables ['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8'] to predict
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection

OzoneDroppedValues_Ozone = OzoneData.dropna(how='any', subset=['Ozone'], inplace=False)
OzoneDroppedValues_Ozone.describe()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
count,116.000000,111.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,42.129310,184.801802,9.862069,77.870690,7.198276,15.534483,0.224138,0.077586,0.224138,0.224138
std,32.987885,91.152302,3.574856,9.485486,1.475715,8.750087,0.418823,0.268680,0.418823,0.418823
min,1.000000,7.000000,2.300000,57.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,113.500000,7.400000,71.000000,6.000000,8.000000,0.000000,0.000000,0.000000,0.000000
50%,31.500000,207.000000,9.700000,79.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000
75%,63.250000,255.500000,11.500000,85.000000,8.250000,22.000000,0.000000,0.000000,0.000000,0.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
# now fill in null values of OzoneData['Ozone'] by predicted values
lm = LinearRegression()
OzoneDroppedValues_Ozone = OzoneData.dropna(how='any', subset=['Ozone'], inplace=False)
X = OzoneDroppedValues_Ozone[['Wind','Temp','Month_5', 'Month_6', 'Month_7','Month_8']]
y = OzoneDroppedValues_Ozone["Ozone"]
lm.fit(X,y)
pvals = feature_selection.f_regression(X,y)[1]
print pvals

# It Seems Month is not significant so I will drop it.
X = OzoneDroppedValues_Ozone[['Wind','Temp']]
y = OzoneDroppedValues_Ozone["Ozone"]
lm.fit(X,y)
pvals = feature_selection.f_regression(X,y)[1]
print pvals

#And now apply it to my dataframe.
X = OzoneData[['Wind','Temp']]
OzoneData["Ozone_predict"] = lm.predict(X)
OzoneData["Ozone"].fillna(value = OzoneData["Ozone_predict"], inplace=True)
OzoneDroppedValues_Ozone.describe()

[  9.27197390e-13   2.93189659e-18   9.47317968e-04   2.31288278e-01
   2.51873571e-03   1.48141562e-03]
[  9.27197390e-13   2.93189659e-18]


,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8,Ozone_predict
count,116.000000,111.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,42.129310,184.801802,9.862069,77.870690,7.198276,15.534483,0.224138,0.077586,0.224138,0.224138,42.129310
std,32.987885,91.152302,3.574856,9.485486,1.475715,8.750087,0.418823,0.268680,0.418823,0.418823,24.877101
min,1.000000,7.000000,2.300000,57.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-22.364061
25%,18.000000,113.500000,7.400000,71.000000,6.000000,8.000000,0.000000,0.000000,0.000000,0.000000,23.296996
50%,31.500000,207.000000,9.700000,79.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,44.216783
75%,63.250000,255.500000,11.500000,85.000000,8.250000,22.000000,0.000000,0.000000,0.000000,0.000000,59.552754
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000,1.000000,1.000000,1.000000,1.000000,94.915959


In [26]:
# now fill in null values of OzoneData['Solar.R'] by predicted values
OzoneDroppedValues_Ozone = OzoneData.dropna(how='any', subset=['Solar.R'], inplace=False)
X = OzoneDroppedValues_Ozone[['Ozone', 'Wind','Temp']]
y = OzoneDroppedValues_Ozone["Solar.R"]
lm.fit(X,y)
pvals = feature_selection.f_regression(X,y)[1]
print pvals

# It Seems Wind is not significant so I will drop it.
X = OzoneDroppedValues_Ozone[['Ozone','Temp']]
y = OzoneDroppedValues_Ozone["Solar.R"]
lm.fit(X,y)
pvals = feature_selection.f_regression(X,y)[1]
print pvals

#And now apply it to my dataframe.
X = OzoneData[['Ozone','Temp']]
OzoneData["Solar_predict"] = lm.predict(X)
OzoneData["Solar.R"].fillna(value = OzoneData["Solar_predict"], inplace=True)
OzoneData.describe()

[  2.49250453e-04   4.95955207e-01   7.51772924e-04]
[ 0.00024925  0.00075177]


,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8,Ozone_predict,Solar_predict
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,42.428551,185.999827,9.941722,78.165563,7.019868,15.801325,0.192053,0.198675,0.205298,0.205298,42.428551,185.999827
std,30.532629,88.654641,3.524984,9.198138,1.406984,8.832531,0.395225,0.400331,0.405263,0.405263,23.925331,27.921933
min,-16.864178,7.000000,1.700000,57.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-22.364061,123.718413
25%,20.000000,119.000000,7.400000,73.000000,6.000000,8.000000,0.000000,0.000000,0.000000,0.000000,26.307740,165.367863
50%,36.000000,203.000000,9.700000,79.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,44.723296,183.125866
75%,59.052378,257.000000,11.500000,85.000000,8.000000,23.000000,0.000000,0.000000,0.000000,0.000000,57.264578,201.794451
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000,1.000000,1.000000,1.000000,1.000000,94.915959,266.139691


#### Now check your filled data - if your predicted values are more than maximum or less than minimum, replace them by max and min

In [50]:
tm = pd.isnull(OzoneData["Ozone"]) == True

False

In [51]:
Ozone_min = OzoneImputeMean["Ozone"].min()
Ozone_max = OzoneImputeMean["Ozone"].max()

Solar_min = OzoneImputeMean["Solar.R"].min()
Solar_max = OzoneImputeMean["Solar.R"].max()

print [Ozone_min, Ozone_max, Solar_min, Solar_max]
for i in range(len(OzoneData)):
    if not i in OzoneData.index:
        continue
    if (OzoneData.loc[i, "Ozone"] < Ozone_min):
        OzoneData.loc[i, "Ozone"] = Ozone_min
    if (OzoneData.loc[i, "Ozone"] > Ozone_max):
        OzoneData.loc[i, "Ozone"] = Ozone_max
    if (OzoneData.loc[i, "Solar.R"] < Solar_min):
        OzoneData.loc[i, "Solar.R"] = Solar_min
    if (OzoneData.loc[i, "Solar.R"] > Solar_max):
        OzoneData.loc[i, "Solar.R"] = Solar_max
OzoneData.describe()    
        


[1.0, 168.0, 7.0, 334.0]


,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8,Ozone_predict,Solar_predict
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,42.618577,185.999827,9.941722,78.165563,7.019868,15.801325,0.192053,0.198675,0.205298,0.205298,42.428551,185.999827
std,30.223265,88.654641,3.524984,9.198138,1.406984,8.832531,0.395225,0.400331,0.405263,0.405263,23.925331,27.921933
min,1.000000,7.000000,1.700000,57.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-22.364061,123.718413
25%,20.000000,119.000000,7.400000,73.000000,6.000000,8.000000,0.000000,0.000000,0.000000,0.000000,26.307740,165.367863
50%,36.000000,203.000000,9.700000,79.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,44.723296,183.125866
75%,59.052378,257.000000,11.500000,85.000000,8.000000,23.000000,0.000000,0.000000,0.000000,0.000000,57.264578,201.794451
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000,1.000000,1.000000,1.000000,1.000000,94.915959,266.139691


## Bonus: In original question, replace missing values of Ozone and Solar.R by Single Imputation Regression Line with Error

In [52]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)
OzoneData.dropna(how = 'all', subset = ['Ozone', 'Solar.R'], inplace = True)
Ozone_min = OzoneData["Ozone"].min()
Ozone_max = OzoneData["Ozone"].max()

Solar_min = OzoneData["Solar.R"].min()
Solar_max = OzoneData["Solar.R"].max()

lm = LinearRegression()
OzoneDroppedValues_Ozone = OzoneData.dropna(how='any', subset=['Ozone'], inplace=False)

X = OzoneDroppedValues_Ozone[['Wind','Temp']]
y = OzoneDroppedValues_Ozone["Ozone"]
lm.fit(X,y)
pvals = feature_selection.f_regression(X,y)[1]
y_hat = lm.predict(X)
standard_error = (sum((y - y_hat) **2)/(len(y)-2))**.5
print standard_error

#And now apply Ozone to my dataframe.
X = OzoneData[['Wind','Temp']]
OzoneData["Ozone_predict"] = lm.predict(X) + np.random.normal(0, standard_error, len(OzoneData))
OzoneData["Ozone"].fillna(value = OzoneData["Ozone_predict"], inplace=True)

#Now Solar.R
OzoneDroppedValues_Ozone = OzoneData.dropna(how='any', subset=['Solar.R'], inplace=False)
X = OzoneDroppedValues_Ozone[['Ozone','Temp']]
y = OzoneDroppedValues_Ozone["Solar.R"]
lm.fit(X,y)
y_hat = lm.predict(X)
standard_error = (sum((y - y_hat) **2)/(len(y)-2))**.5

#And now apply it to my dataframe.
X = OzoneData[['Ozone','Temp']]
OzoneData["Solar_predict"] = lm.predict(X) + np.random.normal(0, standard_error, len(OzoneData))
OzoneData["Solar.R"].fillna(value = OzoneData["Solar_predict"], inplace=True)
OzoneData.describe()

#Finally fix min and max
for i in range(len(OzoneData)):
    if not i in OzoneData.index:
        continue
    if (OzoneData.loc[i, "Ozone"] < Ozone_min):
        OzoneData.loc[i, "Ozone"] = Ozone_min
    if (OzoneData.loc[i, "Ozone"] > Ozone_max):
        OzoneData.loc[i, "Ozone"] = Ozone_max
    if (OzoneData.loc[i, "Solar.R"] < Solar_min):
        OzoneData.loc[i, "Solar.R"] = Solar_min
    if (OzoneData.loc[i, "Solar.R"] > Solar_max):
        OzoneData.loc[i, "Solar.R"] = Solar_max


OzoneData.describe()


21.7588444843


,Ozone,Solar.R,Wind,Temp,Month,Day,Ozone_predict,Solar_predict
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,42.130610,187.417518,9.941722,78.165563,7.019868,15.801325,41.396430,185.299000
std,31.933872,89.303968,3.524984,9.198138,1.406984,8.832531,33.434657,84.688459
min,1.000000,7.000000,1.700000,57.000000,5.000000,1.000000,-37.735846,-32.094097
25%,18.000000,119.000000,7.400000,73.000000,6.000000,8.000000,18.494529,127.538393
50%,34.000000,207.000000,9.700000,79.000000,7.000000,16.000000,44.524041,192.783305
75%,63.029823,258.500000,11.500000,85.000000,8.000000,23.000000,66.615628,235.563799
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000,116.601230,389.402615
